In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sbs
from sklearn.manifold import TSNE
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import MinMaxScaler

In [2]:
import sys
import multiprocessing as mp
from sklearn.neighbors import NearestNeighbors
from math import log
def calculate_weight(pivot, n_1, n_2):
    '''Calculates the weighing factor in numerator and denominator
    This is              1
                  ---------------
                  ||AB|| * ||AC||
    '''
    if type(pivot) != np.ndarray:
        pivot = np.asarray(pivot)
    if type(n_1) != np.ndarray:
        n_1 = np.asarray(n_1)
    if type(n_2) != np.ndarray:
        n_2 = np.asarray(n_2)
    diff_AB = np.subtract(n_1,pivot)
    diff_BC = np.subtract(n_2,pivot)
    return 1.0/(np.linalg.norm(diff_AB)*np.linalg.norm(diff_BC))

def calculate_angle(pivot, n_1, n_2):
    '''Calculates the angular component in the numerator
    This is           <AB,AC>
                -------------------
                ||AB||^2 * ||AC||^2
    '''
    if type(pivot) != np.ndarray:
        pivot = np.asarray(pivot)
    if type(n_1) != np.ndarray:
        n_1 = np.asarray(n_1)
    if type(n_2) != np.ndarray:
        n_2 = np.asarray(n_2)
    diff_AB = np.subtract(n_1,pivot)
    diff_AC = np.subtract(n_2,pivot)
    #dot product is commutative
    if np.linalg.norm(diff_AB)**2 == 0:
        print(diff_AB)
    if np.linalg.norm(diff_AC)**2 == 0:
        print(diff_AC)
    return np.dot(diff_AB,diff_AC)/(np.linalg.norm(diff_AB)**2 * np.linalg.norm(diff_AC)**2)

def perform_selection(X,k):
    '''Variable selection
       Selects the k most explanatory variables
    '''
    return X[:,np.argpartition(np.var(X,axis=0),-k)[-k:]]
    
def abof(D, normalize = False, selective = False, entropy = False, precision = 3):
    '''Full ABOF algorithm.
       Returns: the i/p dataset with an additional ABOF factor column
       Complexity: O(n^3) - implement FAST-ABOF O(n*k^2), LB-ABOF O(n*k^2 + 2n) at cost of accuracy
       Options: normalize - helps to view data better, normalized with max - default:False
                selective - chooses only the top 25% of the columns with highest information/variance - default:False
                entropy   - returns shannon entropy of dataset - default:False
                precision - precision of normalization - default:3
    '''
    #Initialize the ABV array
    sigma_theta = []
    
    #Examine the dataset
    if type(D) != np.ndarray:
        D = np.asarray(D)
    
    #Select only half the (important) variables
    if selective == True:
        D = perform_selection(D,round(D.shape[1]*0.25))
    
    #Iterate over points
    for i in range(len(D)):
        pivot = D[i]
        subset_D_diff_i = np.delete(D,i,0)
        local_theta_n_0 = 0.0
        local_theta_n_1 = 0.0
        local_theta_d = 0.0
        sys.stdout.write("\rComputing datapoint %s...%d%%" % (i+1,round((i+1)/(len(D))*100)))
        sys.stdout.flush()
        for j in range(len(subset_D_diff_i)):
            for k in range(j+1, len(subset_D_diff_i)):
                local_theta_n_0 += calculate_weight(D[i],subset_D_diff_i[j],subset_D_diff_i[k])*(calculate_angle(D[i],subset_D_diff_i[j],subset_D_diff_i[k])**2)
                local_theta_n_1 += calculate_weight(D[i],subset_D_diff_i[j],subset_D_diff_i[k])*calculate_angle(D[i],subset_D_diff_i[j],subset_D_diff_i[k])
                local_theta_d += calculate_weight(D[i],subset_D_diff_i[j],subset_D_diff_i[k])
        sigma_theta.append((local_theta_n_0/local_theta_d) - (local_theta_n_1/local_theta_d)**2)
    
    #Calculate Shannon Entropy
    if entropy == True:
        entr = calc_shannon_ent(D)
        #Stitch the ABV array to i/p matrix
        D = np.hstack((D,np.vstack(sigma_theta)))
        #Normalize
        if normalize == True:
            D = np.round(D/np.max(D),precision)
        return D, entr
    else:
        #Stitch the ABV array to i/p matrix
        D = np.hstack((D,np.vstack(sigma_theta)))
        #Normalize
        if normalize == True:
            D = np.round(D/np.max(D),precision)
        return D
    

def fast_abof(D,k=5, normalize=False):
    '''FAST ABOF algorithm.
       Complexity reduction at expense of accuracy; near quadratic complexity for lower values of k.
       Scales well with larger dimensions and data size. See paper for comparisons with full ABOF.
       Returns: the i/p dataset with an additional ABOF factor column
       Complexity: O(n*k^2) where k <<< n ~ quadratic complexity
    '''
    #Initialize the ABV array
    sigma_theta = []
    
    #Examine the dataset
    if type(D) != np.ndarray:
        D = np.asarray(D)
    
    #Get k nearest neighbors
    neighbors = NearestNeighbors(n_neighbors=k+1, algorithm='ball_tree').fit(D).kneighbors(D)[1]
    
    #Iterate over points
    for i in range(len(D)):
        pivot = D[i]
        local_theta_n_0 = 0.0
        local_theta_n_1 = 0.0
        local_theta_d = 0.0
        sys.stdout.write("\rComputing datapoint %s...%d%%" % (i+1,round((i+1)/(len(D))*100)))
        sys.stdout.flush()
        local_N = neighbors[i][1:]
        for j in range(len(local_N)):
            for k in range(j+1,len(local_N)):
                local_theta_n_0 += calculate_weight(D[i],D[local_N[j]],D[local_N[k]])*(calculate_angle(D[i],D[local_N[j]],D[local_N[k]])**2)
                local_theta_n_1 += calculate_weight(D[i],D[local_N[j]],D[local_N[k]])*calculate_angle(D[i],D[local_N[j]],D[local_N[k]])
                local_theta_d += calculate_weight(D[i],D[local_N[j]],D[local_N[k]])
        if local_theta_d == 0:
            print("Issue:",local_N)
            sigma_theta.append(0)
        else:
            sigma_theta.append((local_theta_n_0/local_theta_d) - (local_theta_n_1/local_theta_d)**2)
            
    #Stitch the ABV array to i/p matrix
    D = np.hstack((D,np.vstack(sigma_theta)))
    
    #Normalize
    if normalize == True:
        D = np.round(D/np.max(D),3)
    
    return D

def calc_shannon_ent(D):
    '''Shannon's entropy for Dataset D given by p*log p
                                                     2
       Returns: entropy(D)
       Complexity: d*log(n)
    '''
    numEntries = len(D)
    labelCounts = {}
    for featVec in D: #the the number of unique elements and their occurance
        currentLabel = featVec[-1]
        if currentLabel not in labelCounts.keys(): labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1
    shannonEnt = 0.0
    for key in labelCounts:
        prob = float(labelCounts[key])/numEntries
        shannonEnt -= prob * log(prob,2) #log base 2
    return shannonEnt